# Ewald Sphere

Play with the Ewald Sphere

This schematics represent the Ewald sphere for a typical high tension of 200 kV and a 1D crystal with lattice parameter 2.5 Angstrom where 100 is a forbiden reflexion, hence the first reflexion is 200. 

The sample and the beam can be tilted between -40 and +40 miliradians

With those (very typical) numbers, the skech is hard to interpret as the angles are small and the reciprocal latice parameter small compared to the wave vector. 

Therefore, the schematics is usualy done in an exagerated way. This can be modelized with the "scaling" parameter. With this parameter, the wave vector is divided by scaling and the beam tilt and sample tilt are multiplied by scaling (in order to keep the represantation of the same diffraction conditions, i.e. a 2 beam case remains a 2-beam case.

You can try to set up a 2-beam case in Bright field (beam tilt =0) by tilting the specimen, and then find the right beam tilt for the dark field condition 

In [1]:
import numpy as np

from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

from bokeh.io import push_notebook, show, output_notebook
from bokeh.plotting import figure
output_notebook()

Loading BokehJS ...

In [2]:
# universal constants
h=6.626*10**(-34) #plank's constant J.s
me=9.109*10**(-31) #electron's mass kg (m_0)
mecsquared=511 # electron's rest mass in keV (m_0*c^2)
electron=1.602*10**(-19)# electron's charge

In [3]:
#electron's wavelength in nm for an acceleration voltage  in kV  (relativistic)
def lamb(Voltage):
    l=10**9*h/np.sqrt(2*me*Voltage*1000*electron)*(1+Voltage/(2*mecsquared))**(-0.5)
    return l

In [4]:
def k(E,scale):
    k = 1/(lamb(E)*scale)
    return k

In [5]:
# Crystal lattice parameter a
# as an example 001 orientation fcc first G is 200 (very typical case)
# We use -2G, -G, 0 G & 2G, 5 beams

Gs = np.array([-3.0, -2.0, -1.0, 0.0, 1.0, 2.0,3.0 ])
a = 0.25 # 0.25nm
recip_x = (2/a)*Gs


In [6]:
# incident wave vector end point depends on scale, E and theta_beam
def ki_endx(E,scale,tilt):
    ki_endx = k(E,scale)*np.sin(scale*tilt*10**(-3))
    return ki_endx

def ki_endy(E,scale,tilt):
    ki_endy = -k(E,scale)*np.cos(scale*tilt*10**(-3))
    return ki_endy

def kix(E,scale,tilt):
    kix = np.array([0,ki_endx(E,scale,tilt)])
    return kix

def kiy(E,scale,tilt):
    kiy = np.array([0,ki_endy(E,scale,tilt)])
    return kiy


In [7]:
def Ewald_angles(scale,tilt):
    ewa = np.linspace(np.pi + ((scale*tilt+400)*10**(-3)),2*np.pi + (scale*tilt-400)*10**(-3),100)
    return ewa

In [8]:
E = 200
scale = 10
beam_tilt = 20
sample_tilt = 20
es = figure(title="Ewald Sphere", plot_height=400, plot_width=800,match_aspect=True)
l0 = es.line([0,0],[k(E,scale)/10,-1.2*k(E,scale)], line_width=1,color="black")
l1 = es.line(k(E,scale)*np.cos(Ewald_angles(scale,beam_tilt)),k(E,scale)*np.sin(Ewald_angles(scale,beam_tilt)),legend_label="Ewald Sphere", line_width=2.5)
l2 = es.line(kix(E,scale,beam_tilt),kiy(E,scale,beam_tilt),legend_label="incident beam", line_width=2.5,color="red")
p3 = es.circle(recip_x*np.cos(sample_tilt*10**(-3)) + ki_endx(E,scale,beam_tilt) ,
               recip_x*np.sin(sample_tilt*10**(-3)) + ki_endy(E,scale,beam_tilt) ,
               legend_label="reciprocal lattice", line_width=3.5,color="green")

es.legend.location = "bottom_left"
#show(es)


def update(scale = 1, beam_tilt = 20, sample_tilt = 20 ):
    l2.data_source.data['x']=kix(E,scale,beam_tilt)
    l2.data_source.data['y']=kiy(E,scale,beam_tilt)
    l1.data_source.data['x']=k(E,scale)*np.cos(Ewald_angles(scale,beam_tilt))
    l1.data_source.data['y']=k(E,scale)*np.sin(Ewald_angles(scale,beam_tilt))
    l0.data_source.data['y']=[k(E,scale)/10,-1.2*k(E,scale)]
    p3.data_source.data['x']=recip_x*np.cos(scale*sample_tilt*10**(-3)) + ki_endx(E,scale,beam_tilt)
    p3.data_source.data['y']=recip_x*np.sin(scale*sample_tilt*10**(-3)) + ki_endy(E,scale,beam_tilt)
   
    
#    l2.data_source.data['y']=eps2(omegap,gamma,omegas)
#    l3.data_source.data['y']=lossfunction(omegap,gamma,omegas)
    push_notebook()
    
es.legend.click_policy="hide"    
show(es, notebook_handle=True)

interact(update, \
         scale=widgets.IntSlider(min=1,max=10,value=1,description="scale"), \
        beam_tilt=widgets.IntSlider(min=-40,max=40,value=20,description="Beam"), \
       sample_tilt=widgets.IntSlider(min=-40,max=40,value=20,description="sample"));



interactive(children=(IntSlider(value=1, description='scale', max=10, min=1), IntSlider(value=20, description=…